In [1]:
from premise import *
import bw2data
from datapackage import Package
bw2data.projects.set_current("ei39")

In [2]:
scenarios = [
    #{"model": "image", "pathway": "SSP2-Base", "year": 2020},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2022},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2025},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2030},
    #{"model": "image", "pathway": "SSP2-Base", "year": 2040},
    {"model": "image", "pathway": "SSP2-Base", "year": 2050},
]

sps = Package("../datapackage.json")

ndb = NewDatabase(
        scenarios=scenarios,
        source_db="ecoinvent 3.9.1 cutoff",
        source_version="3.9.1",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        external_scenarios=[sps,],
        use_multiprocessing=False
)

premise v.(2, 0, 1)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [4]:
ndb.scenarios[0]["external data"][0]

{'production volume': <xarray.DataArray 'value' (region: 1, variables: 64, year: 6)>
 array([[[5.26599105e-06, 9.83804298e-07, 2.52575348e-06, 0.00000000e+00,
          0.00000000e+00, 0.00000000e+00],
         [4.62987274e-06, 6.66775991e-07, 1.84290549e-06, 0.00000000e+00,
          0.00000000e+00, 3.18415645e-08],
         [1.26494770e-05, 2.41887931e-06, 5.78929390e-06, 3.51477849e-03,
          1.06706047e-01, 4.03800037e-01],
         [1.26494490e-05, 2.41887345e-06, 5.78631891e-06, 2.59601637e-05,
          2.14003716e-04, 4.60970592e-05],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          1.19823349e-03, 6.64247532e-03],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00, 7.84586529e-01],
         [9.44999665e-02, 7.94707337e-03, 4.87163804e-02, 0.00000000e+00,
          4.94998570e-04, 1.87920118e-02],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          1.59822393e-

In [3]:
ndb.update("external")

Updating: external:   0%|                       | 0/1 [00:00<?, ?it/s]

Regionalizing electricity production, oil, electricity, high voltage, RoW
not found in regions
Adding electricity production, oil, CH to the database
Regionalizing electricity production, compressed air energy storage, electricity, high voltage, RER
not found in regions
Adding electricity production, compressed air energy storage, CH to the database
Regionalizing heat and power co-generation, natural gas, conventional power plant, 100MW electrical, electricity, high voltage, RoW
not found in regions
Adding heat and power co-generation, natural gas, conventional power plant, 100MW electrical, CH to the database
Regionalizing carbon dioxide, captured at wood burning power plant 20 MW post, pipeline 200km, storage 1000m, carbon dioxide, captured at wood burning power plant 20 MW post, pipeline 200km, storage 1000m, RER
not found in regions
Adding carbon dioxide, captured at wood burning power plant 20 MW post, pipeline 200km, storage 1000m, CH to the database
Regionalizing electricity pro

Updating: external: 100%|███████████████| 1/1 [00:04<00:00,  4.83s/it]

Done!



In [4]:
for ds in ndb.scenarios[0]["database"]:
    if "SPS" in ds["name"]:
        for e in ds["exchanges"]:
            print(e["name"][:40], e["amount"], e.get("location"), e["unit"])
        print()

market for electricity, high voltage (SP 1 CH kilowatt hour
electricity production, hydro, run-of-ri 0.2098645143070873 CH kilowatt hour
electricity production, hydro, reservoir 0.24128864212686676 CH kilowatt hour
electricity production, hydro, pumped st 0.044427203948210384 CH kilowatt hour
electricity production, natural gas, 10M 5.227744089657459e-05 CH kilowatt hour
heat and power co-generation, natural ga 0.000321700668677393 CH kilowatt hour
heat and power co-generation, natural ga 2.404222733862657e-06 CH kilowatt hour
electricity production, at co-generation 1.3897630817602434e-06 CH kilowatt hour
heat and power co-generation, natural ga 0.0007319337772699444 CH kilowatt hour
heat and power co-generation, natural ga 5.906200050120565e-05 CH kilowatt hour
heat and power co-generation, biogas, ga 0.009267434940539356 CH kilowatt hour
electricity production, from incineratio 0.00775758732387819 CH kilowatt hour
electricity production, from incineratio 0.0011658322926844514 CH kil

In [5]:
for ds in ndb.scenarios[0]["database"]:
    if ds["name"] == "electricity supply, high voltage, from vanadium-redox flow battery system" and ds["unit"] == "kilowatt hour":
        print(list(ds.keys()))
        for e in ds["exchanges"]:
            print(e["name"], e["amount"], e.get("location"))
        print()

['location', 'comment', 'production amount', 'source', 'reference product', 'type', 'unit', 'name', 'exchanges', 'worksheet name', 'database', 'code', 'regions', 'technosphere filters', 'biosphere filters', 'production volume variable']
electricity supply, high voltage, from vanadium-redox flow battery system 1 RER
market group for electricity, high voltage 1.33 RER
vanadium-redox flow battery system assembly, 8.3 megawatt hour 2.0408163265306123e-08 RER
vanadium-redox flow battery stack assembly 2.0408163265306123e-08 RER
treatment of stack, for VRFB system -0.00041 RER
treatment of electrolyte tank, for VRFB system -0.00066 RER
treatment of electrolyte solution, for VRFB system -0.00555 RER
treatment of periphericals, for VRFB system -0.0001 RER

['location', 'comment', 'production amount', 'source', 'reference product', 'type', 'unit', 'name', 'exchanges', 'worksheet name', 'database', 'code', 'regions', 'technosphere filters', 'biosphere filters', 'production volume variable', 'pro

In [4]:
ndb.generate_change_report()

Generate change report.
Report saved under /Users/romain/GitHub/sweet_sure-2050-switzerland/dev.


In [6]:
ndb.write_db_to_brightway("test SPS1 7")

Write new database(s) to Brightway.
Running all checks...
Anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:22


Title: Writing activities to SQLite3 database:
  Started: 02/21/2024 12:48:43
  Finished: 02/21/2024 12:49:05
  Total time elapsed: 00:00:22
  CPU %: 95.40
  Memory %: 7.94
Created database: test SPS1 7
Generate scenario report.
Report saved under /Users/romain/GitHub/sweet_sure-2050-switzerland/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/sweet_sure-2050-switzerland/dev.


In [1]:
from premise import *
import bw2data
from datapackage import Package
bw2data.projects.set_current("ei39")
sps = Package("../datapackage.json")
ndb = PathwaysDataPackage(
    scenarios=[
        {"model": "image", "pathway": "SSP2-Base"},
    ],
    #years=[2005, 2010, 2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100],
    years=[2020, 2050],
    source_db="ecoinvent 3.9.1 cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.9", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
    key="tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=",
    external_scenarios=[sps,]
)

premise v.(2, 0, 1)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [2]:
ndb.create_datapackage(
    name="image-SSP2-stem-SPS1",
    contributors=[
        {"name": "Romain",
        "email": "r_s at me.com",}
    ]
)

Updating: trucks: 100%|█████████████████| 1/1 [00:28<00:00, 28.32s/it]


Done!



Updating: external: 100%|███████████████| 1/1 [00:20<00:00, 20.22s/it]


Done!

Extracted 1 worksheets in 0.04 seconds
Remove uncertainty data.
Extracted 1 worksheets in 0.04 seconds
Remove uncertainty data.
Write new database(s) to matrix.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Anomalies found: check the change report.
Matrices saved in /Users/romain/GitHub/sweet_sure-2050-switzerland/dev/pathways/inventories/image/SSP2-Base/2020.
Generate scenario report.
Report saved under /Users/romain/GitHub/sweet_sure-2050-switzerland/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/sweet_sure-2050-switzerland/dev.
{'Carbon Capture|Storage|Direct Air Capture|Cumulative': 'Mt CO2/yr', 'Efficiency|Electricity|Biomass|w/ CCS|1': '-', 'Efficiency|Electricity|Biomass|w/ CCS|2': '-', 'Efficiency|Electricity|Biomass|w/o CCS|1': '-', 'Efficiency|Electricity|Biomass|w/o CCS|2': '-', 'Efficiency|Electricity|Biomass|w/o CCS|3': '-', 'Efficiency|Electricity|Coal|w/ CCS|1': '-', 'Efficiency|E

In [1]:
from pathways import Pathways
p = Pathways(datapackage="image-SSP2-stem-SPS1/datapackage.json")

Invalid datapackage: Descriptor validation error: {'path': 'mapping/mapping.yaml', 'profile': 'data-resource', 'name': 'mapping', 'format': 'yaml', 'mediatype': 'text/yaml', 'encoding': 'utf-8'} is not valid under any of the given schemas at "resources/17" in descriptor and at "properties/resources/items/oneOf" in profile
Invalid datapackage: Descriptor validation error: 'data-resource' is not one of ['tabular-data-resource'] at "resources/17/profile" in descriptor and at "properties/resources/items/properties/profile/enum" in profile
Variable Final Energy|Industry excl Non-Energy Use|Food and Tobacco|Electricity not found in scenario data.
Variable Final Energy|Industry excl Non-Energy Use|Food and Tobacco|Gases|Natural Gas not found in scenario data.
Variable Final Energy|Industry excl Non-Energy Use|Food and Tobacco|Heat not found in scenario data.
Variable Final Energy|Industry excl Non-Energy Use|Food and Tobacco|Hydrogen not found in scenario data.
Variable Final Energy|Industry 

In [2]:
import numpy as np
p.calculate(
    methods=[
        'IPCC 2021 - climate change - GWP 100a, incl. H',
        'IPCC 2021 - climate change - GWP 100a, incl. H and bio CO2',
    ],
    regions=["CH",],
    scenarios=[
        "SSP2-Base",
        #"SSP2-RCP19",
    ],
    variables=[
        "hydro_run_of_river",
        "hydro (reservoir)",
        "hydro (pumped storage)",
        "nuclear (boiling water)",
        "nuclear (pressure water)",
        "oil (peak devices)",
        "oil (DH CHP)",
        "oil (industry CHP)",
        "gas (CCGT)",
        "gas (OCGT)",
        "gas (CCGT CCS)",
        "gas (CHP DH)",
        "gas (fuel cell DH)",
        "gas (CHP industry)",
        "gas (CHP CCS industry)",
        "gas (CHP fuel cell reformer)",
        "gas (CHP single family)",
        "gas (single family CHP fuel cell)",
        "gas (multi family CHP)",
        "gas (multi family CHP fuel cell)",
        "biogas (services CHP)",
        "gas (services CHP)",
        "gas (services CHP fuel cell)",
        "waste incineration, fossil",
        "waste incineration, fossil (CHP)",
        "waste incineration, fossil (industrial)",
        "waste incineration, fossil (CCS)",
        "waste incineration, fossil (CHP, CCS)",
        "waste incineration, fossil (industrial, CCS)",
        "waste incineration, non-fossil",
        "waste incineration, non-fossil (CHP)",
        "waste incineration, non-fossil (industrial)",
        "waste incineration, non-fossil (CCS)",
        "waste incineration, non-fossil (CHP, CCS)",
        "waste incineration, non-fossil (industrial, CCS)",
        "pv (large scale)",
        "pv (industry)",
        "pv (services)",
        "pv residential (multi family)",
        "pv residential (single family)",
        "wind",
        "geothermal",
        "hydrogen fuel cell (DH CHP)",
        "hydrogen fuel cell (industrial CHP)",
        "hydrogen fuel cell (services CHP)",
        "biomass (DH)",
        "biomass (CHP)",
        "biomass (CCS, BECCS)",
        "biomass (CCS, industrial)",
        "battery (large)",
        "battery (medium)",
        "battery (industry)",
        "battery (services)",
        "battery (single family)",
        "battery (multi family)",
        "CAES",
        "import",
    ],
    years=[
        2020,
        2030,
        2040,
        2050
    ],
    characterization=True,
    data_type=np.float32,
    multiprocessing=False,
    demand_cutoff=0.001,
)
arr = p.display_results()

Calculating LCA results for image...
--- Calculating LCA results for SSP2-Base...
------ Calculating LCA results for 2020...
CH is not an IAM location.
CH is not an IAM location.
CH is not an IAM location.


0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


------ Calculating LCA results for 2030...
CH is not an IAM location.
CH is not an IAM location.
CH is not an IAM location.


0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00


------ Calculating LCA results for 2040...
CH is not an IAM location.
CH is not an IAM location.
CH is not an IAM location.


0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00


------ Calculating LCA results for 2050...
CH is not an IAM location.
CH is not an IAM location.
CH is not an IAM location.


0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00


In [3]:
df = arr.to_dataframe("x").unstack()["x"].reset_index().melt(id_vars=[
    "model", "scenario", "region", "impact_category", "variable", "year"
])

In [4]:
df=df.loc[~df["value"].isnull()]

In [5]:
len(df)

6726

In [6]:
from pivottablejs import pivot_ui
from IPython.display import HTML
pivot_ui(df, outfile_path='pivottable_ch.html')

In [6]:
from pivottablejs import pivot_ui
from IPython.display import HTML
pivot_ui(df, outfile_path='pivottable_ch.html')